In [17]:
import os
import sys

import matplotlib.pyplot as plt

module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

import numpy as np
import pandas as pd
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
import xgboost as xgb

import warnings
warnings.filterwarnings('ignore')

# my own module
from conf.configure import Configure
from utils import data_util

In [2]:
import numpy as np
import pandas as pd
import datetime
from sklearn.cross_validation import KFold
from sklearn.cross_validation import train_test_split
import time
#%matplotlib inline
from sklearn import preprocessing
from sklearn.preprocessing import Imputer
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import ShuffleSplit
from sklearn.metrics import make_scorer, mean_squared_error, r2_score
from sklearn.linear_model import Ridge, LassoCV,LassoLarsCV, ElasticNet
from sklearn.kernel_ridge import KernelRidge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from scipy.stats import skew
from time import time

R2 = make_scorer(r2_score, greater_is_better=True)

In [3]:
print 'load datas...'
train, test = data_util.load_dataset()

y_train_all = train['y']
del train['ID']
del train['y']
id_test = test['ID']
del test['ID']

load datas...


In [4]:
X_train = train
X_test = test
y_train = y_train_all
y_mean = np.mean(y_train)
del train, test, y_train_all

In [19]:
xgb_params = {
    'eta': .005,
    'max_depth': 2,
    'subsample': 0.95,
    'reg_alpha': 0,
    'base_score': y_mean,
    'min_child_weight':6,
    'colsample_bytree': 0.55,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1,
    'booster' :'gbtree',
    'tuneLength': 3,
    'seed': 123
}

dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test)

cv_output = xgb.cv(xgb_params, dtrain, num_boost_round=1000, early_stopping_rounds=50,\
    verbose_eval=50, show_stdv=False)
cv_output[['train-rmse-mean', 'test-rmse-mean']].plot()

dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test)

num_boost_rounds = len(cv_output)
model = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round=num_boost_rounds)

y_predict = model.predict(dtest)

[0]	train-rmse:12.6415	test-rmse:12.6389
[50]	train-rmse:11.1931	test-rmse:11.1965
[100]	train-rmse:10.1987	test-rmse:10.2063
[150]	train-rmse:9.54607	test-rmse:9.55825
[200]	train-rmse:9.10687	test-rmse:9.12412
[250]	train-rmse:8.82474	test-rmse:8.84976
[300]	train-rmse:8.63685	test-rmse:8.66969
[350]	train-rmse:8.50951	test-rmse:8.55405
[400]	train-rmse:8.41913	test-rmse:8.48062
[450]	train-rmse:8.35144	test-rmse:8.43056
[500]	train-rmse:8.29716	test-rmse:8.3926
[550]	train-rmse:8.25092	test-rmse:8.36588
[600]	train-rmse:8.20874	test-rmse:8.34704
[650]	train-rmse:8.17015	test-rmse:8.3299
[700]	train-rmse:8.13405	test-rmse:8.31785
[750]	train-rmse:8.10022	test-rmse:8.30829
[800]	train-rmse:8.06867	test-rmse:8.30293
[850]	train-rmse:8.03862	test-rmse:8.29791
[900]	train-rmse:8.01135	test-rmse:8.29603
[950]	train-rmse:7.98647	test-rmse:8.29361


In [20]:
fig, ax = plt.subplots(1, 1, figsize=(8, 13))
xgb.plot_importance(model, max_num_features=50, height=0.5, ax=ax)

In [12]:
xgb_params = {'learning_rate': 0.005,
              'max_depth': 4,
              'subsample': 0.95,
              'objective': 'reg:linear',
              #'eval_metric': 'rmse',
              'silent': 1,
              'reg_alpha': 0,
              'min_child_weight':4,
              'colsample_bytree': 0.95,
              'gamma':.4,
              #'booster' :'gbtree',
              #'tuneLength': 3,
              'seed': 5 }
#print(len(cv_output)) #309

xgb_params['n_estimators']=616 #len(cv_output)

model = XGBRegressor(**xgb_params)
model.fit(X_train, y_train)
y_predict = model.predict(X_test)

GradientBoostingModel

In [10]:
model = GradientBoostingRegressor(learning_rate=0.001, loss="huber", max_depth=3, max_features=0.55, random_state=5,
                                  min_samples_leaf=18, min_samples_split=14, subsample=0.7)
model.fit(X_train, y_train)
y_predict = model.predict(X_test)

In [21]:
output = pd.DataFrame({'ID': id_test, 'y': y_predict})
output.to_csv('../result/jul2_5.csv', index=False)